In [1]:
import pyvo
import download_kepler_metadata as dl

In [2]:
base_service_url = "https://exoplanetarchive.ipac.caltech.edu/TAP"
service = pyvo.dal.TAPService(base_service_url)

koi_df = dl.get_confirmed_koi(service)
tce_df = dl.get_exoplanet_tces(service)
non_ttv_df = dl.get_non_ttv_planets(service)
pl_name_df = dl.get_pl_names(service)

In [3]:
print(koi_df.colnames)
print(tce_df.colnames)
print(non_ttv_df.colnames)
print(pl_name_df.colnames)

['kepid', 'kepoi_name', 'kepler_name', 'koi_tce_plnt_num', 'koi_count', 'koi_comment', 'koi_period', 'koi_time0bk', 'koi_duration', 'koi_model_snr', 'koi_quarters', 'koi_num_transits']
['kepid', 'tce_plnt_num', 'tce_time0', 'tce_period', 'tce_time0bk', 'tce_duration', 'tce_incl', 'tce_num_transits', 'tce_model_snr', 'tce_depth']
['pl_name', 'ttv_flag']
['kepid', 'koi_name', 'kepler_name', 'pl_name']


In [7]:
import pandas as pd 

koi_pd = koi_df.to_pandas()
tce_pd = tce_df.to_pandas()
ttv_pd = non_ttv_df.to_pandas()
pl_name_pd = pl_name_df.to_pandas()

koi_filtered = (
    koi_pd[~koi_pd["koi_tce_plnt_num"].isna()]
    .rename(columns={"koi_tce_plnt_num": "tce_plnt_num"})
)

koi_tce = koi_filtered.merge(tce_pd, how="inner", on=["kepid", "tce_plnt_num"])
# koi_tce

# merge pl_names into ttv to get kep id's
non_ttv = pl_name_pd.merge(ttv_pd, how="inner", on="pl_name")

non_ttv.sort_values("kepid")

,kepid,koi_name,kepler_name,pl_name,ttv_flag
110,10000941,K04146.01,Kepler-1558 b,Kepler-1558 b,0
2320,10001368,K02785.01,Kepler-1344 b,Kepler-1344 b,0
468,10002866,K00723.01,Kepler-222 b,Kepler-222 b,0
2042,10004519,K01572.01,Kepler-894 b,Kepler-894 b,0
2333,10004738,K01598.02,Kepler-310 d,Kepler-310 d,0
...,...,...,...,...,...
1098,9967771,K01875.01,Kepler-990 b,Kepler-990 b,0
2083,9973109,K02018.01,Kepler-1056 b,Kepler-1056 b,0
575,9992325,K02205.01,Kepler-1140 b,Kepler-1140 b,0
1808,9995402,K02832.01,Kepler-1361 b,Kepler-1361 b,0


In [ ]:
from astropy.table import Table
# Table.write.list_formats()
Table.write.help("csv")

In [ ]:
tce_table_name = "q1_q17_dr25_tce"
base_service_url = "https://exoplanetarchive.ipac.caltech.edu/TAP"

service = pyvo.dal.TAPService(base_service_url)

adql_query = """
    SELECT pl_name, pl_masse, pl_rade
    FROM ps
    WHERE upper(soltype) LIKE '%CONF%'
    AND pl_masse BETWEEN 0.5 AND 2.0
"""

results = service.search(adql_query)

result_table = results.to_table()
result_table

In [ ]:
def query_exoplanet_archive(request: str):
    base_url = "https://exoplanetarchive.ipac.caltech.edu/cgi-bin/nstedAPI/nph-nstedAPI?"
    request_url = base_url + request

In [ ]:
def mast_query(request):
    """Perform a MAST query.
 
        Parameters
        ----------
        request (dictionary): The MAST request json object
 
        Returns head,content where head is the response HTTP headers, and content is the returned data"""
 
    # Base API url
    request_url='https://mast.stsci.edu/api/v0/invoke'
 
    # Grab Python Version
    version = ".".join(map(str, sys.version_info[:3]))
 
    # Create Http Header Variables
    headers = {"Content-type": "application/x-www-form-urlencoded",
               "Accept": "text/plain",
               "User-agent":"python-requests/"+version}
 
    # Encoding the request as a json string
    req_string = json.dumps(request)
    req_string = urlencode(req_string)
 
    # Perform the HTTP request
    resp = requests.post(request_url, data="request="+req_string, headers=headers)
 
    # Pull out the headers and response content
    head = resp.headers
    content = resp.content.decode('utf-8')
 
    return head, content


In [ ]:
def download_request(payload, filename, download_type="file"):
    request_url='https://mast.stsci.edu/api/v0.1/Download/' + download_type
    resp = requests.post(request_url, data=payload)
 
    with open(filename,'wb') as FLE:
        FLE.write(resp.content)
 
    return filename

In [ ]:
request = {
    'service':'Mast.Missions.List',
    'params':{},
    'format':'json'
}

headers, out_string = mast_query(request)

out_data = [x['distinctValue'] for x in json.loads(out_string)['data']]

out_data

In [ ]:
print(json.loads(out_string)["data"])